In [1]:
import yfinance as yf
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# For time stamps
from datetime import datetime
from scikeras.wrappers import KerasRegressor, KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping
from keras.constraints import MaxNorm


In [2]:
#iShares MSCI World Index ETF
data = yf.download('AAPL', start='2012-01-01', end=datetime.now())
#data = yf.download('AAPL', period='4y')
df = pd.DataFrame(data)
df = df['Adj Close']
df_shifted = df.shift().dropna().values
df = df.values
trend_values = [j - i for i, j in zip(df[1:], df_shifted)]
trend_values = [(float(value),) for value in trend_values]

[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['AAPL']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2012-01-01 -> 2024-03-08 19:36:06.688897)')


In [35]:
import os
import random
import numpy as np
import tensorflow as tf
import glob
from matplotlib import pyplot as plt
import matplotlib

GPU_STRING = '/gpu:0'
MODEL_NAME = "Grid-Search Batch-Size & Epochs"
EPOCHS = 10
STEPS_PER_EPOCH = 10
VALIDATION_STEPS = 32
SEQ_LEN_PAST = 62
SEQ_LEN_FUTURE = 1
NUM_INPUT_PARAMETERS = 1
NUM_OUTPUT_PARAMETERS = 1

class ModelHistory(tf.keras.callbacks.Callback):
  def __init__(self, model_path):
    self.model_path = model_path
    self.loss = []
    self.loss_val = []
    self.mae = []
    self.mae_val = []
    self.mse = []
    self.mse_val = []

  def on_epoch_end(self, batch, logs):
    self.loss.append(logs.get('loss'))
    self.loss_val.append(logs.get('val_loss'))
    self.mae.append(logs.get('mean_absolute_error'))
    self.mse.append(logs.get('mean_squared_error'))
    self.mae_val.append(logs.get('val_mean_absolute_error'))
    self.mse_val.append(logs.get('val_mean_squared_error'))
    self.plot_data()


  def plot_data(self):
    vis_path = os.path.join(self.model_path, 'vis')

    if not os.path.exists(vis_path):
      os.makedirs(vis_path)

    model_name = self.model_path.split('/')[-1]

    plt.clf()
    plt.plot(self.loss)
    plt.plot(self.loss_val)
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.savefig(os.path.join(vis_path, 'loss.png'))

def select_data(values):
  selected_inputs = []
  selected_labels = []

  num = 0
  while num < len(values) - SEQ_LEN_PAST:
    sub_seq_input = values[num:num + SEQ_LEN_PAST]
    trend_val = values[num + SEQ_LEN_PAST][0]
    if (trend_val >= 0):
      sub_seq_label = 1
    else:
      sub_seq_label = 0

    selected_inputs.append(sub_seq_input)
    selected_labels.append(sub_seq_label)

    num += 1

  return np.asarray(selected_inputs).astype(float), np.asarray(selected_labels).squeeze().astype(float)

def setup_model_lstm(dropout_rate, weight_constraint):
  input = tf.keras.layers.Input(shape=(SEQ_LEN_PAST, NUM_INPUT_PARAMETERS), name='input')

  dr = dropout_rate
  x = tf.keras.layers.LSTM(8, return_sequences=True, kernel_constraint=MaxNorm(weight_constraint))(input)
  x = tf.keras.layers.Dropout(dr)(x)
  x = tf.keras.layers.LSTM(16, kernel_constraint=MaxNorm(weight_constraint))(x)
  x = tf.keras.layers.Dropout(dr)(x)
  x = tf.keras.layers.Dense(SEQ_LEN_FUTURE * NUM_OUTPUT_PARAMETERS, activation='sigmoid')(x)
  #x = tf.keras.layers.Reshape((SEQ_LEN_FUTURE, NUM_OUTPUT_PARAMETERS))(x)

  model = tf.keras.models.Model(input, x)
  opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.005)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])
  model.summary()
  return model

def setup_model_mlp():
  input = tf.keras.layers.Input(shape=(SEQ_LEN_PAST, NUM_INPUT_PARAMETERS), name='input')

  dp = 0.4
  x = tf.keras.layers.Flatten()(input)
  x = tf.keras.layers.Dense(256, activation='relu')(x)
  x = tf.keras.layers.Dropout(dp)(x)
  x = tf.keras.layers.Dense(256, activation='relu')(x)
  x = tf.keras.layers.Dropout(dp)(x)
  x = tf.keras.layers.Dense(256, activation='relu')(x)
  x = tf.keras.layers.Dropout(dp)(x)
  x = tf.keras.layers.Dense(32, activation='relu')(x)
  x = tf.keras.layers.Dropout(dp)(x)
  x = tf.keras.layers.Dense(32, activation='relu')(x)
  x = tf.keras.layers.Dropout(dp)(x)  
  x = tf.keras.layers.Dense(SEQ_LEN_FUTURE * NUM_OUTPUT_PARAMETERS, activation='sigmoid')(x)
  #x = tf.keras.layers.Reshape((SEQ_LEN_FUTURE, NUM_OUTPUT_PARAMETERS))(x)

  model = tf.keras.models.Model(input, x)
  opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])
  model.summary()
  return model

def setup_model_conv_1d():
  input = tf.keras.layers.Input(shape=(SEQ_LEN_PAST, NUM_INPUT_PARAMETERS), name='input')

  dp = 0.4
  x = tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', padding='same')(input)
  x = tf.keras.layers.Dropout(dp)(x)
  x = tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', padding='same')(x)
  x = tf.keras.layers.Dropout(dp)(x)
  x = tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', padding='same')(x)
  x = tf.keras.layers.Dropout(dp)(x)  
  x = tf.keras.layers.GlobalAveragePooling1D()(x)

  x = tf.keras.layers.Dense(64, activation='relu')(x)  
  x = tf.keras.layers.Dense(SEQ_LEN_FUTURE * NUM_OUTPUT_PARAMETERS, activation='sigmoid')(x)
  #x = tf.keras.layers.Reshape((SEQ_LEN_FUTURE, NUM_OUTPUT_PARAMETERS))(x)

  model = tf.keras.models.Model(input, x)
  opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])
  model.summary()
  return model 

def train(data_path, model_path, model, from_checkpoint=False):

  X, Y = select_data(trend_values)
  model_history_callback = ModelHistory(model_path)

  # Set up early stopping
  early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

  batch_size = [32, 64, 128]
  epochs = [10, 20, 50]
  param_grid = dict(batch_size=batch_size, epochs=epochs)
  grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
  grid_result = grid.fit(X, Y, verbose=1, callbacks=[model_history_callback])

  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  for mean, stdev, param in zip(means, stds, params):
      print("%f (%f) with: %r" % (mean, stdev, param))

def run():
  physical_devices = tf.config.list_physical_devices('GPU')
  print("\n")
  print("GPUs:", physical_devices)
  print("\n")

  with tf.device(GPU_STRING):
    path = 'zeitreihen/'
    data_path = path + 'datasets/sin_data/'
    model_path = path + 'models/' + MODEL_NAME + '/'

    if not os.path.exists(model_path):
      os.makedirs(model_path)

    # create model
    model = KerasClassifier(model=setup_model_mlp, verbose=1)

    train(data_path, model_path, model)

if __name__== "__main__":
  run()



GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]




KeyboardInterrupt: 